<a href="https://colab.research.google.com/github/bosunKwak/BigData/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Overall Process
<img width="380" alt="스크린샷 2022-04-19 오후 11 29 44" src="https://user-images.githubusercontent.com/87002218/164027812-736c6c74-e20b-4fcc-a057-dc2c1524d2c2.png">


"here is the example"
1) Tokenize : here, is , the ,example
- 문장을 토큰 시퀀스로 분석하는 과정
- 토큰의 분리기준은 다를 수 있지만 문장을 구성하는 기본단위 -ex) 단어

2) Vocabulary : 475, 21, 2, 30, 5297
- 모은 단어의 토큰들을 합쳐 하나의 list를 만듦(어휘 집합 == 토크나이즈 수행 후 중복을 제거한 토큰들의 집합)
- 각 단어를 문자 단위 ID 시퀀스로 변환하기 위해 사용

3) Embedding : [[0.81,0.27,0.59], [ , , ], [ , , ], [ , , ]]
- 컴퓨터는 인간이 사용하는 자연어를 있는 그대로 이해하는 것이 아니라 숫자를 계산
- 사람이 쓰는 자연어를 기계가 이해할 수 잇는 숫자의 나열인 벡터로 바꾼 결과/ 일련의 과정 전체
- 단어나 문장을 각각 벡터로 변환해 벡터 공간(vector space)로 끼워넣는다(embed) 

# Text classification
- [pytorch를 이용해서 모델을 개발하는 전반적인 과정](https://github.com/bosunKwak/BigData/blob/main/overall_process.ipynb)을 text data를 가지고 실습
- [tutorial 참고](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)
- data preparation & model development 
- [dataset](https://pytorch.org/text/stable/datasets.html#ag-news)

## 1. Data Preparation

### - Access to the raw dataset iterators

In [1]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = iter(AG_NEWS(split='train'))
# iter : 순환 가능한 객체로 만드는 것

29.5MB [00:00, 49.9MB/s]


In [2]:
next(train_iter)
# 하나씩 꺼내옴

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# tokenizer 선정 
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

# tokenizer 수행하는 함수
def yield_tokens(data_iter):
  # (category , text) : 뒤에 있는 text만 가져옴 
    for _, text in data_iter:
        yield tokenizer(text) # text를 tokenizer로 token화

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"]) # tokenize된 token들이 vocab에 들어오고, 새로운 단어의 경우 <unknown>처리
vocab.set_default_index(vocab["<unk>"]) # set_default_index : unknown 이라는 token을 index로 지정하겠다는 의미

In [4]:
vocab.lookup_token(4)

'to'

In [5]:
vocab(['here','is','an','example','to'])

[475, 21, 30, 5297, 4]

### - Prepare data processing pipelines
text가 들어왔을 때, 이것을 어떻게 가공할 것인지에 대한 절차 정의

In [6]:
# text를 받았을 때 처리하는 pipeline
text_pipeline = lambda x: vocab(tokenizer(x)) # x를 받아 tokenizer를 통해 토큰으로 바꾸고 다시 vocab으로

# label 정보를 받았을 때 처리하는 pipeline
label_pipeline = lambda x: int(x) - 1 # 1~4 -> 0~3

In [7]:
text_pipeline('here is an example')

[475, 21, 30, 5297]

### - Generate data batch and iterator


collate_fn 
- batch를 통해 collate_fn로 전달되면 데이터를 처리
- collate : 종이를 순서에 맞춰서 정렬을 하는 것 


In [8]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# batch : Dataloader에서 받아온 batch 
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0] #offsets : 문장이 시작되는 지점 (첫번째 문장의 offset : 0 )
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))  
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64) 
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) # cumsum : 누적해서 더함(0,3,5,2)->(0,3,8,10) offse[:-1] : 마지막 빼고 계산
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch) # collate_fn에 지정 

In [9]:
next(iter(dataloader))
# vocab.lookup_token(1)

(tensor([2, 2, 2, 2, 2, 2, 2, 2]),
 tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
            27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
         67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
          1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
           797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
           564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
            28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
             1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
            14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
             8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
           294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
             1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
            27,  

### Model Development

#### Define the model
Embedding(EmbeddingBag) layer & Linear Layer
- EmbeddingBag : Embedding 값들의 평균을 내어주는 것

In [10]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

#### Initiate an instance

In [11]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [12]:
print(vocab_size)

95811


#### Define functions to train the model and evaluate results


In [13]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

#### Split the dataset and run the model

dataset - train set(vali, train), test set 으로 split

In [14]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# split
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

1.86MB [00:00, 53.8MB/s]                  


| epoch   1 |   500/ 1782 batches | accuracy    0.680
| epoch   1 |  1000/ 1782 batches | accuracy    0.853
| epoch   1 |  1500/ 1782 batches | accuracy    0.878
-----------------------------------------------------------
| end of epoch   1 | time: 17.32s | valid accuracy    0.878 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.896
| epoch   2 |  1000/ 1782 batches | accuracy    0.898
| epoch   2 |  1500/ 1782 batches | accuracy    0.904
-----------------------------------------------------------
| end of epoch   2 | time: 15.62s | valid accuracy    0.894 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.914
| epoch   3 |  1000/ 1782 batches | accuracy    0.914
| epoch   3 |  1500/ 1782 batches | accuracy    0.914
-----------------------------------------------------------
| end of epoch   3 | time: 16.85s | valid accuracy    0.903 
-------------------------------

#### Evaluate the model with test dataset

In [15]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

test accuracy    0.908


#### Test on a random news

In [16]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
